In [5]:
#!aws s3 cp s3://cv-data-pavanpaj/Text_to_Image_GAN_AWS.py .
# !aws s3 cp s3://cv-data-pavanpaj/data_util.py .

download: s3://cv-data-pavanpaj/data_util.py to ./data_util.py 


In [8]:
import logging
logging.getLogger("sagemaker").setLevel(logging.ERROR)

In [11]:
import sagemaker
from sagemaker.pytorch import PyTorch

s3_bucket = "cv-data-pavanpaj"

# Define SageMaker Training Job
estimator = PyTorch(
    entry_point="Text_to_Image_GAN_AWS.py",  # Main training script
    source_dir="scripts",  
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version="1.9",
    py_version="py38",
    hyperparameters={
        "epochs": 1,
        "batch_size": 64,
        "embed_dim": 1024,
        "embed_out_dim": 128,
        "l1_coef": 50,
        "l2_coef": 100,
        "num_workers": 0,
        "log_interval": 5,
        "s3_bucket": s3_bucket,
        "s3_dataset": "birds_small.hdf5",
        "s3_model_path": "saved_models",
        "s3_image_path": "generated_images"
    },
    output_path=f"s3://{s3_bucket}/model_output/"
)

# Start Training Job
estimator.fit()


2025-02-28 22:59:03 Starting - Starting the training job...
..25-02-28 22:59:17 Starting - Preparing the instances for training.
..25-02-28 22:59:36 Downloading - Downloading input data.
....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2025-02-28 23:01:13,167 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-02-28 23:01:13,171 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-02-28 23:01:13,181 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2025-02-28 23:01:13,184 sagemaker_pytorch_container.training INFO     Invoking user training script.
2025-02-28 23:01:13,352 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-02-28 23:01:13,362 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-02-28 23:01:13,373 sagemaker-training-tool

In [12]:
import boto3

s3 = boto3.client("s3")
s3_bucket = "cv-data-pavanpaj"

# List saved models
response = s3.list_objects_v2(Bucket=s3_bucket, Prefix="saved_models/")
for obj in response.get("Contents", []):
    print(obj["Key"])

saved_models/discriminator_epoch_000.pth
saved_models/discriminator_epoch_001.pth
saved_models/discriminator_final.pth
saved_models/generator_epoch_000.pth
saved_models/generator_epoch_001.pth
saved_models/generator_final.pth
saved_models/model.tar.gz
